In [2]:
import numpy as np
import matplotlib.pyplot as plt

from desdeo_problem import ScalarObjective, MOProblem, variable_builder

from smt.sampling_methods import LHS

var_names = ['a', 'b', 'c', 'd']
initial_values = [1,1,1,1] # ?
lower_bounds =  np.array([0.125, 0.1, 0.1, 0.125])
upper_bounds = np.array([5.0, 10.0, 10.0, 5.0])
variables = variable_builder(var_names, initial_values, lower_bounds, upper_bounds)

print('length of variables', len(variables))

def f_1(x):
    return 1.10471 * x[0] ** 2 * x[1] + 0.04811 * x[2] * x[3] * (14.0 + x[1])

def f_2(x):
    return 2.1952 / (x[3] * x[2] ** 3)


obj1 = ScalarObjective('f_1', f_1)
obj2 = ScalarObjective('f_2', f_2)


problem = MOProblem(objectives=[obj1, obj2], variables=variables)


length of variables 4


In [3]:
xlimits = np.array([lower_bounds, upper_bounds, lower_bounds, upper_bounds])
sampling = LHS(xlimits=xlimits)

num = 100
x = sampling(num)

print(x[0,0])

# x = x / np.linalg.norm(x)

print(x[0,0])
print(x.shape)

0.10287500000000001
0.10287500000000001
(100, 4)


In [4]:
y = []

for xi in x:
    y.append([f_1(xi), f_2(xi)])
    
y = np.array(y)



print(y[0])

[  0.6862969  271.06650725]


In [21]:
# import GPy

# kernel = GPy.kern.RBF(input_dim=4)

# gpy_model_y_1 = GPy.models.GPRegression(x, y, kernel)
# #gpy_model_y_2 = GPy.models.GPRegression(x, y[:,1], kernel) # Throws error ?

# print(gpy_model_y_1)

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.metrics import r2_score, mean_squared_error


kernel = DotProduct() + WhiteKernel()

gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(x, y)

gpr.score(x, y)

gpr_predict = gpr.predict(x, return_std=True)

gpr_r_2 = r2_score(y_true=y, y_pred=gpr_predict[0])
print('Kriging R-squared', gpr_r_2)


gpr_rmse = mean_squared_error(y_true=y, y_pred=gpr_predict[0], squared=False)
print('Kriging RMSE', gpr_rmse)

Kriging R-squared 0.14418563368249793
Kriging RMSE 28.785903336917848


In [25]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

forest = RandomForestRegressor(max_depth=2, random_state=0)
forest.fit(x, y)

forest_predict = forest.predict(x)

print(forest_predict.shape)

forest_r_2 = r2_score(y_true=y, y_pred=forest_predict)
print('Random forest R-squared', forest_r_2)


forest_rmse = mean_squared_error(y_true=y, y_pred=forest_predict, squared=False)
print('Random forest RMSE', forest_rmse)

(100, 2)
Random forest R-squared 0.7744654923210998
Random forest RMSE 10.81454249244477


In [43]:
# TEST

num_test = 50
x_test = sampling(num_test)

y_test = gpr.sample_y(x_test)

print(y_test[:,:,0].shape)

gpr_r_2 = r2_score(y_true=y[:50], y_pred=y_test[:,:,0])
print('Test - Kriging R-squared', gpr_r_2)


gpr_rmse = mean_squared_error(y_true=y[:50], y_pred=y_test[:,:,0], squared=False)
print('Test - Kriging RMSE', gpr_rmse)

(50, 2)
Kriging R-squared -29721.54781786479
Kriging RMSE 58.81285041241952
